In [22]:
import glob
import tensorflow as tf

tflite_model_file = './tflite_models/model.tflite'

In [23]:
import tensorflow_io as tfio
from tensorflow.python.ops import gen_audio_ops as audio_ops
import tensorflow.experimental.numpy as tnp

def generate_spectrogram(file_path):
    # does not scale as sniffing should be the same independent of background level
    audio_tensor = tfio.audio.AudioIOTensor(file_path, dtype=tf.int16)
    audio = tf.cast(audio_tensor[:], tf.float32)
    spectrogram = audio_ops.audio_spectrogram(audio,
                                              window_size=320,
                                              stride=160,
                                              magnitude_squared=True)
    spectrogram = tf.nn.pool(
        input=tf.expand_dims(spectrogram, -1),
        window_shape=[1, 6],
        strides=[1, 6],
        pooling_type='AVG',
        padding='SAME')
    spectrogram = tf.squeeze(spectrogram, axis=0)
    # Not sure whether the log is a good idea...
    spectrogram = tnp.log10(spectrogram + 1e-6)
    return spectrogram

In [59]:
# prepare tfMicro Interpreter
interpreter = tf.lite.Interpreter(model_path="./tflite_models/model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_index = input_details[0]['index']
output_index = output_details[0]['index']

In [60]:
# make a single based on tflite prediction
def make_tflite_prediction(one_second_file):
    spectrogram = generate_spectrogram(one_second_file)
    interpreter.set_tensor(input_index, [spectrogram])
    interpreter.invoke()
    tflite_result = interpreter.get_tensor(output_index)
    return tflite_result

In [73]:
# make predictions for all files
false_positive = 0
false_negative = 0
true_positive = 0
true_negative = 0
for audio_file in glob.glob("./data/test/Background/*.wav"):
    tf_lite_result = make_tflite_prediction(audio_file)[0]
    if tf_lite_result[1] > 0.5:
        true_negative += 1
    else:
        false_positive +=1

for audio_file in glob.glob("./data/test/Sniffing/*.wav"):
    tf_lite_result = make_tflite_prediction(audio_file)[0]
    if tf_lite_result[0] > 0.5:
        true_positive += 1
    else:
        false_negative +=1

tp_rate = true_positive / (true_positive + false_negative) * 100
tn_rate = true_negative / (true_negative + false_positive) * 100
fn_rate = false_negative / (true_negative + true_positive) * 100
fp_rate = false_positive / (true_positive + true_negative) * 100
print(f"TP: {true_positive} ({tp_rate})\tFP: {false_positive}")
print(f"FN: {false_negative}\tTN: {true_negative} ({tn_rate})")

TP: 46 (71.875)	FP: 66
FN: 18	TN: 126 (65.625)
